# NBA Predictive Machine Learning Model
## Jordan Stapinski (jstapins), Calvin Lui (clui)
### Project II - Practical Data Science 67-364 Spring 2018

### Conventions on Representations
For the process of numerically encoding our statistics to be used in a machine learning model, we use the following numerical conventions (more concretely defined in cell one of code)

#### Conferences
| Conference | Numerical Representation |
|------------|--------------------------|
| Eastern    | 0                        |
| Western    | 1                        |
#### Divisions
| Division     | Numerical Representation |
|--------------|--------------------------|
| Atlantic     | 0                        |
| Southeastern | 1                        |
| Central      | 2                        |
| Northwestern | 3                        |
| Southwestern | 4                        |
| Pacific      | 5                        |
#### Locations
| Location | Numerical Representation |
|----------|--------------------------|
| Away     | 0                        |
| Home     | 1                        |
#### Results
| Result   | Numerical Representation |
|----------|--------------------------|
| Loss     | 0                        |
| Win      | 1                        |
#### Season Types
| Season Type    | Numerical Representation |
|----------------|--------------------------|
| Preseason      | 0                        |
| Regular Season | 1                        |
| Postseason     | 2                        |
#### Roles
| Role     | Numerical Representation |
|----------|--------------------------|
| Bench    | 0                        |
| Starter  | 1                        |
#### Positions
| Position       | Numerical Representation |
|----------------|--------------------------|
| Point Guard    | 0                        |
| Guard          | 1                        |
| Shooting Guard | 2                        |
| Small Forward  | 3                        |
| Forward        | 4                        |
| Power Forward  | 5                        |
| Center         | 6                        |

In [239]:
# code used to create TEAM_DOUBLED_CSV_FILE
# creates additional row to represent away team perspective
# necessary for group by operations

'''
def duplicate_row(row):
    new_row = row.to_frame().T
    for column in new_row.columns:
        if 'team' in column:
            oppt_value = new_row[column]
            team_value = new_row[column.replace('team', 'oppt')]
            new_row[column.replace('team', 'oppt')] = oppt_value
            new_row[column] = team_value
            
    # add home/away reversed row to team_stats
    team_stats.append(new_row)
    
    # keep orignal row unchanged
    return row

team_stats = team_stats.apply(duplicate_row, axis=1)
team_stats.head(10)
'''

"\ndef duplicate_row(row):\n    new_row = row.to_frame().T\n    for column in new_row.columns:\n        if 'team' in column:\n            oppt_value = new_row[column]\n            team_value = new_row[column.replace('team', 'oppt')]\n            new_row[column.replace('team', 'oppt')] = oppt_value\n            new_row[column] = team_value\n            \n    # add home/away reversed row to team_stats\n    team_stats.append(new_row)\n    \n    # keep orignal row unchanged\n    return row\n\nteam_stats = team_stats.apply(duplicate_row, axis=1)\nteam_stats.head(10)\n"

In [240]:
# Data Importing
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd

%matplotlib inline

# Defining Constants
PLAYER_CSV_FILE = './nba-enhanced-stats/2017-18_playerBoxScore.csv'
TEAM_CSV_FILE = './nba-enhanced-stats/2017-18_teamBoxScore.csv'
TEAM_DOUBLED_CSV_FILE = './nba-enhanced-stats/2017-18_teamBoxScore_doubled.csv'
PREVIEW_LEN = 10

# Manual Categorical Mappings for Logical Encoding
teamConf_numeric = {"East": 0, "West": 1}
teamDiv_numeric = {"Atlantic": 0, "Southeast": 1, "Central": 2, "Northwest": 3, "Southwest": 4, "Pacific": 5}
teamLoc_numeric = {"Away": 0, "Home": 1}
teamRslt_numeric = {"Loss": 0, "Win": 1}
seasTyp_numeric = {"Pre": 0, "Regular": 1, "Post": 2}
playStat_numeric = {"Bench": 0, "Starter": 1}
playPos_numeric = {"PG": 0, "G": 1, "SG": 2, "SF": 3, "F": 4, "PF": 5, "C": 6}
# Team Name and Player Name will be One-Hot Encoded

# Possible Columns to Remove for Avoiding Multicollinearity
derived_team_player_cols = ['teamTREB%', 'teamASST%', "teamTS%", "teamEFG%", "teamOREB%", "teamDREB%", "teamTO%", "teamSTL%", "teamBLK%", "teamBLKR", "teamPPS", "teamFIC", "teamFIC40", "teamOrtg", "teamDrtg", "teamEDiff", "teamPlay%", "teamAR", "teamAST/TO", "teamSTL/TO"]
derived_oppt_player_cols = list(map(lambda x: x.replace("team", "oppt"), derived_team_player_cols))

# Referee Names to be Removed
useless_cols = ['offLNm1', 'offFNm1', 'offLNm2', 'offFNm2', 'offLNm3', 'offFNm3']

player_cols = ['gmDate', 'gmTime', 'seasTyp', 'playLNm', 'playFNm', 'teamAbbr', 'teamConf', 'teamDiv', 'teamLoc', 'teamRslt', 'teamDayOff', 'offLNm1', 'offFNm1', 'offLNm2', 'offFNm2', 'offLNm3', 'offFNm3', 'playDispNm', 'playStat', 'playMin', 'playPos', 'playHeight', 'playWeight', 'playBDate', 'playPTS', 'playAST', 'playTO', 'playSTL', 'playBLK', 'playPF', 'playFGA', 'playFGM', 'playFG%', 'play2PA', 'play2PM', 'play2P%', 'play3PA', 'play3PM', 'play3P%', 'playFTA', 'playFTM', 'playFT%', 'playORB', 'playDRB', 'playTRB', 'opptAbbr', 'opptConf', 'opptDiv', 'opptLoc', 'opptRslt', 'opptDayOff']
team_cols = ['gmDate', 'gmTime', 'seasTyp', 'offLNm1', 'offFNm1', 'offLNm2', 'offFNm2', 'offLNm3', 'offFNm3', 'teamAbbr', 'teamConf', 'teamDiv', 'teamLoc', 'teamRslt', 'teamMin', 'teamDayOff', 'teamPTS', 'teamAST', 'teamTO', 'teamSTL', 'teamBLK', 'teamPF', 'teamFGA', 'teamFGM', 'teamFG%', 'team2PA', 'team2PM', 'team2P%', 'team3PA', 'team3PM', 'team3P%', 'teamFTA', 'teamFTM', 'teamFT%', 'teamORB', 'teamDRB', 'teamTRB', 'teamPTS1', 'teamPTS2', 'teamPTS3', 'teamPTS4', 'teamPTS5', 'teamPTS6', 'teamPTS7', 'teamPTS8', 'teamTREB%', 'teamASST%', 'teamTS%', 'teamEFG%', 'teamOREB%', 'teamDREB%', 'teamTO%', 'teamSTL%', 'teamBLK%', 'teamBLKR', 'teamPPS', 'teamFIC', 'teamFIC40', 'teamOrtg', 'teamDrtg', 'teamEDiff', 'teamPlay%', 'teamAR', 'teamAST/TO', 'teamSTL/TO', 'opptAbbr', 'opptConf', 'opptDiv', 'opptLoc', 'opptRslt', 'opptMin', 'opptDayOff', 'opptPTS', 'opptAST', 'opptTO', 'opptSTL', 'opptBLK', 'opptPF', 'opptFGA', 'opptFGM', 'opptFG%', 'oppt2PA', 'oppt2PM', 'oppt2P%', 'oppt3PA', 'oppt3PM', 'oppt3P%', 'opptFTA', 'opptFTM', 'opptFT%', 'opptORB', 'opptDRB', 'opptTRB', 'opptPTS1', 'opptPTS2', 'opptPTS3', 'opptPTS4', 'opptPTS5', 'opptPTS6', 'opptPTS7', 'opptPTS8', 'opptTREB%', 'opptASST%', 'opptTS%', 'opptEFG%', 'opptOREB%', 'opptDREB%', 'opptTO%', 'opptSTL%', 'opptBLK%', 'opptBLKR', 'opptPPS', 'opptFIC', 'opptFIC40', 'opptOrtg', 'opptDrtg', 'opptEDiff', 'opptPlay%', 'opptAR', 'opptAST/TO', 'opptSTL/TO', 'poss', 'pace']

player_stats = pd.read_csv(PLAYER_CSV_FILE, sep=',', names=player_cols, encoding='latin-1', skiprows=[0])
team_stats = pd.read_csv(TEAM_DOUBLED_CSV_FILE, sep=',', names=team_cols, encoding='latin-1', skiprows=[0])

### Data Cleaning

Performing some data cleaning according to the above conventions, as well as removing dashes and colon in dates and times.

Furthermore, we get the stats for each team quickly before training the model.

In [241]:
# Data Cleaning for team_stats

# Date and Time Formatting
team_stats['gmDate'] = team_stats['gmDate'].apply(lambda x: x.replace("-", ""))
team_stats['gmTime'] = team_stats['gmTime'].apply(lambda x: x.replace(":", ""))

# Categorical Encoding
team_stats['teamConf'] = team_stats['teamConf'].apply(lambda x: teamConf_numeric[x])
team_stats['opptConf'] = team_stats['opptConf'].apply(lambda x: teamConf_numeric[x])
team_stats['teamDiv'] = team_stats['teamDiv'].apply(lambda x: teamDiv_numeric[x])
team_stats['opptDiv'] = team_stats['opptDiv'].apply(lambda x: teamDiv_numeric[x])
team_stats['teamLoc'] = team_stats['teamLoc'].apply(lambda x: teamLoc_numeric[x])
team_stats['opptLoc'] = team_stats['opptLoc'].apply(lambda x: teamLoc_numeric[x])
team_stats['teamRslt'] = team_stats['teamRslt'].apply(lambda x: teamRslt_numeric[x])
team_stats['opptRslt'] = team_stats['opptRslt'].apply(lambda x: teamRslt_numeric[x])
team_stats['seasTyp'] = team_stats['seasTyp'].apply(lambda x: seasTyp_numeric[x])

# Drop Useless Columns
team_stats = team_stats.drop(useless_cols, axis=1)

# Remove Duplicate Rows
# team_stats = team_stats.iloc[::2]

In [242]:
# Data Cleaning for player_stats

# Date and Time Formatting
player_stats['gmDate'] = player_stats['gmDate'].apply(lambda x: x.replace("-", ""))
player_stats['gmTime'] = player_stats['gmTime'].apply(lambda x: x.replace(":", ""))
player_stats['playBDate'] = player_stats['gmDate'].apply(lambda x: x.replace("-", ""))

# Categorical Encoding
player_stats['teamConf'] = player_stats['teamConf'].apply(lambda x: teamConf_numeric[x])
player_stats['opptConf'] = player_stats['opptConf'].apply(lambda x: teamConf_numeric[x])
player_stats['teamDiv'] = player_stats['teamDiv'].apply(lambda x: teamDiv_numeric[x])
player_stats['opptDiv'] = player_stats['opptDiv'].apply(lambda x: teamDiv_numeric[x])
player_stats['teamLoc'] = player_stats['teamLoc'].apply(lambda x: teamLoc_numeric[x])
player_stats['opptLoc'] = player_stats['opptLoc'].apply(lambda x: teamLoc_numeric[x])
player_stats['teamRslt'] = player_stats['teamRslt'].apply(lambda x: teamRslt_numeric[x])
player_stats['opptRslt'] = player_stats['opptRslt'].apply(lambda x: teamRslt_numeric[x])
player_stats['seasTyp'] = player_stats['seasTyp'].apply(lambda x: seasTyp_numeric[x])
player_stats['playStat'] = player_stats['playStat'].apply(lambda x: playStat_numeric[x])
player_stats['playPos'] = player_stats['playPos'].apply(lambda x: playPos_numeric[x])

# Drop Useless Columns
player_stats = player_stats.drop(useless_cols, axis=1)

### One-Hot Encoding
Algorithm-friendly data formatting for NBA Teams & Players.

In [243]:
# One-Hot Encoding for Players
one_hot_player = pd.get_dummies(player_stats['playDispNm'])
player_stats = player_stats.join(one_hot_player)

# Drop Processed Columns
player_stats = player_stats.drop(['playDispNm', 'playLNm', 'playFNm'], axis=1)

In [244]:
# Create Table for Use Later
# Team Representation based on Players

# join team_stats & player_stats
join_cols = ['gmDate', 'seasTyp', 'gmTime', 'teamAbbr', 'teamConf', 'teamDiv', 'teamLoc', 'teamRslt', 'teamDayOff', 'opptAbbr', 'opptConf', 'opptDiv', 'opptLoc', 'opptRslt', 'opptDayOff']
joined_stats = pd.merge(team_stats, player_stats, how='left', left_on=join_cols, right_on=join_cols, suffixes=('_t', '_p'))

func = {}
for column in joined_stats.columns:
    if column in team_cols:
        func[column] = 'first'
    elif column in player_cols:
        # drop player related statistics
        pass
    else:
        # check whether player is playing for home team
        func[column] = 'max'

# group by game + team
grp = joined_stats.groupby(['gmDate', 'teamAbbr']).agg(func)

grp.head(PREVIEW_LEN)

gmDate gmTime  seasTyp teamAbbr  teamConf  teamDiv  \
gmDate   teamAbbr                                                         
20171017 BOS       20171017   0800        1      BOS         0        0   
         CLE       20171017   0800        1      CLE         0        2   
         GS        20171017   1030        1       GS         1        5   
         HOU       20171017   1030        1      HOU         1        4   
20171018 ATL       20171018   0830        1      ATL         0        1   
         BKN       20171018   0700        1      BKN         0        0   
         BOS       20171018   0730        1      BOS         0        0   
         CHA       20171018   0700        1      CHA         0        1   
         DAL       20171018   0830        1      DAL         1        4   
         DEN       20171018   0900        1      DEN         1        3   

                   teamLoc  teamRslt  teamMin  teamDayOff  teamPTS  teamAST  \
gmDate   teamAbbr                                                             
20171017 BOS             0         0      241           0       99       24   
         CLE             1         1      240           0      102       19   
         GS              1         0      241           0      121       34   
         HOU             0         1      239           0      122       28   
20171018 ATL             0         1      241           0      117       20   
         BKN             0         0      241           0      131       22   
         BOS             1         0      240           0      100       23   
         CHA             0         0      238           0       90       16   
         DAL             1         0      240           0      111       27   
         DEN             0         0      242           0       96       21   

                   teamTO  teamSTL  teamBLK  teamPF  teamFGA  teamFGM  \
gmDate   teamAbbr                                                       
20171017 BOS           12       11        4      24       88       36   
         CLE           17        3        4      25       83       38   
         GS            17        5        9      25       80       43   
         HOU           13        9        5      16       97       47   
20171018 ATL           13       11        6      18       94       48   
         BKN           20        7        2      25       94       45   
         BOS           15       12        2      27       91       39   
         CHA           17        4        3      15       73       29   
         DAL           16        5        4      19       86       38   
         DEN           22        4        3      18       77       36   

                   teamFG%  team2PA  team2PM  team2P%  team3PA  team3PM  \
gmDate   teamAbbr                                                         
20171017 BOS        0.4091       56       28   0.5000       32        8   
         CLE        0.4578       61       33   0.5410       22        5   
         GS         0.5375       50       27   0.5400       30       16   
         HOU        0.4845       56       32   0.5714       41       15   
20171018 ATL        0.5106       76       39   0.5132       18        9   
         BKN        0.4787       64       33   0.5156       30       12   
         BOS        0.4286       63       28   0.4444       28       11   
         CHA        0.3973       43       20   0.4651       30        9   
         DAL        0.4419       41       21   0.5122       45       17   
         DEN        0.4675       50       23   0.4600       27       13   

                   team3P%  teamFTA  teamFTM  teamFT%  teamORB  teamDRB  \
gmDate   teamAbbr                                                         
20171017 BOS        0.2500       25       19   0.7600        9       37   
         CLE        0.2273       25       21   0.8400        9       41   
         GS         0.5333       21       19   0.9048        6       35   
         HOU      

In [245]:
# One-Hot Encoding for Teams

# team_stats
team_stats['teamAbbr'] = team_stats['teamAbbr'].apply(lambda x: "team_" + x)
one_hot_team = pd.get_dummies(team_stats['teamAbbr'])
team_stats = team_stats.join(one_hot_team)

team_stats['opptAbbr'] = team_stats['opptAbbr'].apply(lambda x: "oppt_" + x)
one_hot_oppt_team = pd.get_dummies(team_stats['opptAbbr'])
team_stats = team_stats.join(one_hot_oppt_team)

# player_stats
player_stats['teamAbbr'] = player_stats['teamAbbr'].apply(lambda x: "team_" + x)
one_hot_team = pd.get_dummies(player_stats['teamAbbr'])
player_stats = player_stats.join(one_hot_team)

player_stats['opptAbbr'] = player_stats['opptAbbr'].apply(lambda x: "oppt_" + x)
one_hot_oppt_team = pd.get_dummies(player_stats['opptAbbr'])
player_stats = player_stats.join(one_hot_oppt_team)

# Drop Processed Columns
team_stats = team_stats.drop(['teamAbbr', 'opptAbbr'], axis=1)
player_stats = player_stats.drop(['teamAbbr', 'opptAbbr'], axis=1)

In [246]:
team_stats.head(PREVIEW_LEN)

,gmDate,gmTime,seasTyp,teamConf,teamDiv,teamLoc,teamRslt,teamMin,teamDayOff,teamPTS,teamAST,teamTO,teamSTL,teamBLK,teamPF,teamFGA,teamFGM,teamFG%,team2PA,team2PM,team2P%,team3PA,team3PM,team3P%,teamFTA,teamFTM,teamFT%,teamORB,teamDRB,teamTRB,teamPTS1,teamPTS2,teamPTS3,teamPTS4,teamPTS5,teamPTS6,teamPTS7,teamPTS8,teamTREB%,teamASST%,teamTS%,teamEFG%,teamOREB%,teamDREB%,teamTO%,teamSTL%,teamBLK%,teamBLKR,teamPPS,teamFIC,teamFIC40,teamOrtg,teamDrtg,teamEDiff,teamPlay%,teamAR,teamAST/TO,teamSTL/TO,opptConf,opptDiv,opptLoc,opptRslt,opptMin,opptDayOff,opptPTS,opptAST,opptTO,opptSTL,opptBLK,opptPF,opptFGA,opptFGM,opptFG%,oppt2PA,oppt2PM,oppt2P%,oppt3PA,oppt3PM,oppt3P%,opptFTA,opptFTM,opptFT%,opptORB,opptDRB,opptTRB,opptPTS1,opptPTS2,opptPTS3,opptPTS4,opptPTS5,opptPTS6,opptPTS7,opptPTS8,opptTREB%,opptASST%,opptTS%,opptEFG%,opptOREB%,opptDREB%,opptTO%,opptSTL%,opptBLK%,opptBLKR,opptPPS,opptFIC,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace,team_ATL,team_BKN,team_BOS,team_CHA,team_CHI,team_CLE,team_DAL,team_DEN,team_DET,team_GS,team_HOU,team_IND,team_LAC,team_LAL,team_MEM,team_MIA,team_MIL,team_MIN,team_NO,team_NY,team_OKC,team_ORL,team_PHI,team_PHO,team_POR,team_SA,team_SAC,team_TOR,team_UTA,team_WAS,oppt_ATL,oppt_BKN,oppt_BOS,oppt_CHA,oppt_CHI,oppt_CLE,oppt_DAL,oppt_DEN,oppt_DET,oppt_GS,oppt_HOU,oppt_IND,oppt_LAC,oppt_LAL,oppt_MEM,oppt_MIA,oppt_MIL,oppt_MIN,oppt_NO,oppt_NY,oppt_OKC,oppt_ORL,oppt_PHI,oppt_PHO,oppt_POR,oppt_SA,oppt_SAC,oppt_TOR,oppt_UTA,oppt_WAS
0,20171017,0800,1,0,0,0,0,241,0,99,24,12,11,4,24,88,36,0.4091,56,28,0.5000,32,8,0.2500,25,19,0.7600,9,37,46,19,19,33,28,0,0,0,0,47.9167,66.6667,0.5000,0.4545,18.0000,80.4348,10.8108,10.9692,3.9888,7.1429,1.1250,75.375,62.5519,98.7227,101.7143,-2.9916,0.3956,17.7778,2.0000,91.6667,0,2,1,1,240,0,102,19,17,3,4,25,83,38,0.4578,61,33,0.5410,22,5,0.2273,25,21,0.8400,9,41,50,29,25,18,30,0,0,0,0,52.0833,50.0000,0.5426,0.4880,18.0000,82.0000,15.3153,2.9916,3.9888,6.5574,1.2289,66.625,55.5208,101.7143,98.7227,2.9916,0.4176,14.6154,1.1176,17.6471,100.2809,99.8648,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20171017,0800,1,0,2,1,1,240,0,102,19,17,3,4,25,83,38,0.4578,61,33,0.5410,22,5,0.2273,25,21,0.8400,9,41,50,29,25,18,30,0,0,0,0,52.0833,50.0000,0.5426,0.4880,19.5652,82.0000,15.3153,2.9916,3.9888,6.5574,1.2289,66.625,55.5208,101.7143,98.7227,2.9916,0.4176,14.6154,1.1176,17.6471,0,0,0,0,241,0,99,24,12,11,4,24,88,36,0.4091,56,28,0.5000,32,8,0.2500,25,19,0.7600,9,37,46,19,19,33,28,0,0,0,0,47.9167,66.6667,0.5000,0.4545,19.5652,80.4348,10.8108,10.9692,3.9888,7.1429,1.1250,75.375,62.5519,98.7227,101.7143,-2.9916,0.3956,17.7778,2.0000,91.6667,100.2809,100.2809,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20171017,1030,1,1,4,0,1,239,0,122,28,13,9,5,16,97,47,0.4845,56,32,0.5714,41,15,0.3659,19,13,0.6842,10,33,43,34,28,26,34,0,0,0,0,51.1905,59.5745,0.5790,0.5619,22.2222,84.6154,10.9834,8.7795,4.8775,8.9286,1.2577,97.875,81.9038,119.0108,118.0353,0.9755,0.4700,19.1309,2.1538,69.2308,1,5,1,0,241,0,121,34,17,5,9,25,80,43,0.5375,50,27,0.5400,30,16,0.5333,21,19,0.9048,6,35,41,35,36,30,20,0,0,0,0,48.8095,79.0698,0.6779,0.6375,14.6341,85.3659,16.0015,4.8775,8.7795,18.0000,1.5125,103.875,86.2033,118.0353,119.0108,-0.9755,0.4725,24.2442,2.0000,29.4118,102.5117,102.9406,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20171017,1030,1,1,5,1,0,241,0,121,34,17,5,9,25,80,43,0.5375,50,27,0.5400,30,16,0.5333,21,19,0.9048,6,35,41,35,36,30,20,0,0,0,0,48.8095,79.0698,0.6779,0.6375,15.3846,77.7778,16.0015,4.8775,8.7795,18.0000,1.5125,103.875,86.2033,118.0353,119.0108,-0.9755,0.4725,24.2442,2.0000,29.4118,1,4,0,1,239,0,122,28,13,9,5,16,97,47,0.4845,56,32,0.5714,41,15,0.3659,19,13,0.6842,10,33,43,34,28,26,34,0,0,0,0,51.1905,59.5745,0.5790,0.5619,23.2558,76.7

In [247]:
player_stats.head(PREVIEW_LEN)

,gmDate,gmTime,seasTyp,teamConf,teamDiv,teamLoc,teamRslt,teamDayOff,playStat,playMin,playPos,playHeight,playWeight,playBDate,playPTS,playAST,playTO,playSTL,playBLK,playPF,playFGA,playFGM,playFG%,play2PA,play2PM,play2P%,play3PA,play3PM,play3P%,playFTA,playFTM,playFT%,playORB,playDRB,playTRB,opptConf,opptDiv,opptLoc,opptRslt,opptDayOff,Aaron Brooks,Aaron Gordon,Aaron Harrison,Aaron Jackson,Abdel Nader,Adreian Payne,Al Horford,Al Jefferson,Al-Farouq Aminu,Alan Williams,Alec Burks,Alec Peters,Alex Caruso,Alex Len,Alex Poythress,Alfonzo McKinnie,Allen Crabbe,Amir Johnson,Andre Drummond,Andre Iguodala,Andre Ingram,Andrew Bogut,Andrew Harrison,Andrew White,Andrew Wiggins,AndrÃ© Roberson,Ante Å½iÅ¾iÄ,Anthony Brown,Anthony Davis,Anthony Tolliver,Antonio Blakeney,Antonius Cleveland,Aron Baynes,Arron Afflalo,Austin Rivers,Avery Bradley,Bam Adebayo,Ben McLemore,Ben Moore,Ben Simmons,Bismack Biyombo,Blake Griffin,Boban MarjanoviÄ,Bobby Brown,Bobby Portis,Bogdan Bogdanovic,Bojan BogdanoviÄ,Bradley Beal,Brandan Wright,Brandon Ingram,Brandon Jennings,Brandon Paul,Briante Weber,Brice Johnson,Brook Lopez,Bruno Caboclo,Bryn Forbes,Buddy Hield,C.J. McCollum,C.J. Miles,C.J. Williams,Caleb Swanigan,Cameron Payne,Caris LeVert,Carmelo Anthony,Cedi Osman,Chandler Parsons,Channing Frye,Charles Cooke,Cheick Diallo,Chinanu Onuaku,Chris Boucher,Chris McCullough,Chris Paul,Clint Capela,Cody Zeller,Cole Aldrich,Corey Brewer,Cory Joseph,Courtney Lee,Cristiano FelÃ­cio,D'Angelo Russell,D.J. Augustin,D.J. Wilson,Dakari Johnson,Damian Jones,Damian Lillard,Damien Wilkins,Damion Lee,Damyean Dotson,Daniel Hamilton,Daniel Theis,Danilo Gallinari,Danny Green,Dante Cunningham,Dante Exum,Danuel House,Dario Saric,Darius Miller,Darrell Arthur,Darren Collison,Darrun Hilliard,David Nwaba,David Stockton,David West,Davon Reed,De'Aaron Fox,DeAndre Jordan,DeAndre Liggins,DeAndreâ Bembry,DeMar DeRozan,DeMarcus Cousins,DeMarre Carroll,Dejounte Murray,Delon Wright,Demetrius Jackson,Dennis SchrÃ¶der,Dennis Smith,Denzel Valentine,Derrick Favors,Derrick Jones,Derrick Rose,Derrick Walton,Derrick White,Derrick Williams,Devin Booker,Devin Harris,Devin Robinson,Dewayne Dedmon,Deyonta Davis,Dillon Brooks,Dion Waiters,Dirk Nowitzki,Domantas Sabonis,Donovan Mitchell,Dorian Finney-Smith,Doug McDermott,Dragan Bender,Draymond Green,Dwayne Bacon,Dwight Buycks,Dwight Howard,Dwight Powell,Dwyane Wade,DÄvis BertÄns,E'Twaun Moore,Ed Davis,Edmond Sumner,Ekpe Udoh,Elfrid Payton,Emeka Okafor,Emmanuel Mudiay,Enes Kanter,Eric Bledsoe,Eric Gordon,Eric Moreland,Erik McCree,Ersan Ilyasova,Evan Fournier,Evan Turner,Frank Kaminsky,Frank Mason,Frank Ntilikina,Fred VanVleet,Furkan Korkmaz,Garrett Temple,Gary Harris,Gary Payton II,George Hill,Georges Niang,Georgios Papagiannis,Gerald Green,Gian Clavell,Giannis Antetokounmpo,Glenn Robinson,Goran DragiÄ,Gordon Hayward,Gorgui Dieng,Greg Monroe,Guerschon Yabusele,Harrison Barnes,Hassan Whiteside,Henry Ellenson,Ian Clark,Ian Mahinmi,Ike Anigbogu,Iman Shumpert,Isaiah Canaan,Isaiah Hicks,Isaiah Taylor,Isaiah Thomas,Isaiah Whitehead,Ishmael Smith,Ivan Rabb,Ivica Zubac,J.J. Redick,J.R. Smith,JaKarr Sampson,JaMychal Green,JaVale McGee,Jabari Bird,Jabari Parker,Jack Cooley,Jacob Pullen,Jacob Wiley,Jae Crowder,Jahlil Okafor,Jake Layman,Jakob PÃ¶ltl,Jalen Jones,Jamal Crawford,Jamal Murray,Jameel Warney,Jameer Nelson,Jamel Artis,James Ennis,James Harden,James Johnson,James Michael McAdoo,James Web III,James Young,Jamil Wilson,Jared Dudley,Jarell Eddie,Jarell Martin,Jarrett Allen,Jarrett Jack,Jason Smith,Jason Terry,Jawun Evans,Jaylen Brown,Jaylen Morris,Jayson Tatum,Jeff Green,Jeff Teague,Jeff Withey,Jerami Grant,Jeremy Evans,Jeremy Lamb,Jeremy Lin,Jerian Grant,Jerryd Bayless,Jimmy Butler,Joakim Noah,Jodie Meeks,Joe Harris,Joe Ingles,Joe Johnson,Joel Bolomboy,Joel Embiid,Joffrey Lauvergne,John Collins,John Henson,John Holland,John Wall,Johnathan Motley,Johnny O'Bryant,Jon Leuer,Jonas Jerebko,Jonas ValanÄiÅ«nas,Jonathan Gibson,Jonathan Isaac,Jonathon Simmons,Jordan 

## Baseline Predictive Model
Using a Gradient Boosting Classifier, we achieve a baseline of about 66% accuracy on game outcomes by using the teams playing as our only features. We really wish to bump this metric up as much as possible by better dissecting team and player statistics and then composing hypothetical scenarios not tested in the season.

Note this model doesn't really use any team or player statistics. Later, we will represent the team as a composition of individucal players, which should help in improving our ability to predict games.

In [248]:
# Predicting Wins only based on Teams Playing
# Can Build Another Model to Predict Points per Team
X = team_stats.drop(list(set(team_cols) - set(useless_cols) - set(['teamAbbr', 'opptAbbr'])), axis=1)
y = team_stats.teamRslt

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.63821138211382111

## Representing the Player
For representing a single player, we are going to leverage the object-oriented functionality of Python and construct a `Player` class. A player is represented with the following attributes:
- `name` (the player's name)
- `ppg` (points per game)
- `apg` (assists per game)
- `rpg` (rebounds per game)
- `spg` (steals per game)
- `bpg` (blocks per game)
- `fgapg` (field goals attempted per game)
- `fgp` (field goal percentage)
- `ftapg` (free throws attempted per game)
- `ftp` (free throw percentage)
- `gp` (games played)
- `raw_player_df` (dataframe for just this player)

and methods:
- `to_training_data`: returns a vectorized set of features for this player
- `stats_vs_team(team_abbr)`: returns players vectorized stats vs a specific team

In [249]:
class Player(object):
    def __init__(self, name, ppg, apg, rpg, spg, bpg, fgapg, fgp, ftapg, ftp, gp, raw_player_df):
        self.name = name
        self.ppg = ppg
        self.apg = apg
        self.rpg = rpg
        self.spg = spg
        self.bpg = bpg
        self.fgapg = fgapg
        self.fgp = fgp
        self.ftapg = ftapg
        self.ftp = ftp
# Debating use of below metric
#         self.tpp = tpp
        self.gp = gp
# Debating use of below metric
#         self.mpg = mpg
        self.raw_player_df = raw_player_df
        
    def stats_vs_team(self, team_abbr):
        specific_team = self.raw_player_df[self.raw_player_df['oppt_' + team_abbr] == 1]
        ppg = calculate_ppg(specific_team)
        apg = calculate_apg(specific_team)
        trb = calculate_trb(specific_team)
        spg = calculate_spg(specific_team)
        bpg = calculate_bpg(specific_team)
        fgapg = calculate_fgapg(specific_team)
        fgp = calculate_fgp(specific_team)
        ftapg = calculate_ftapg(specific_team)
        ftp = calculate_ftp(specific_team)
        gp = len(specific_team)
        
        return {'ppg':ppg, 'apg':apg, 'trb':trb, 'spg':spg, 'bpg':bpg, 'fgapg':fgapg, 'fgp':fgp, 'ftapg':ftapg, 'ftp':ftp, 'gp':gp}
        
    def to_training_data(self):
        # Note: Leaving out games played
        return [self.ppg, self.apg, self.rpg, self.spgm, self.bpg, self.fgapg, self.fgp, self.ftapg, self.ftp]

# Get List of players
num_non_player_cols = 100
players_registered = player_stats.columns.tolist()[num_non_player_cols:]

# Helper functions to calculate statistics (per game, per season for certain ones)
def calculate_per_game(stat, specific_player):
    total_games = len(specific_player)
    return stat / total_games   

def calculate_ppg(specific_player):
    total_points = specific_player['playPTS'].sum()
    return calculate_per_game(total_points, specific_player)

def calculate_apg(specific_player):
    total_assists = specific_player['playAST'].sum()
    return calculate_per_game(total_assists, specific_player)

def calculate_trb(specific_player):
    total_rebounds = specific_player['playTRB'].sum()
    return calculate_per_game(total_rebounds, specific_player)

def calculate_spg(specific_player):
    total_steals = specific_player['playSTL'].sum()
    return calculate_per_game(total_steals, specific_player)

def calculate_bpg(specific_player):
    total_blocks = specific_player['playBLK'].sum()
    return calculate_per_game(total_blocks, specific_player)

# Field goal attempts per game
def calculate_fgapg(specific_player):
    total_fga = specific_player['playFGA'].sum()
    return calculate_per_game(total_fga, specific_player)

# Field goals percentage made per game
def calculate_fgp(specific_player):
    return specific_player['playFG%'].mean()

# Free throw attempts per game
def calculate_ftapg(specific_player):
    total_fta = specific_player['playFTA'].sum()
    return calculate_per_game(total_fta, specific_player)

# Free throw percentage per game
def calculate_ftp(specific_player):
    return specific_player['playFT%'].mean()

all_stats = {}
for player in players_registered:
    specific_player = player_stats[player_stats[player] == 1]
    ppg = calculate_ppg(specific_player)
    apg = calculate_apg(specific_player)
    trb = calculate_trb(specific_player)
    spg = calculate_spg(specific_player)
    bpg = calculate_bpg(specific_player)
    fgapg = calculate_fgapg(specific_player)
    fgp = calculate_fgp(specific_player)
    ftapg = calculate_ftapg(specific_player)
    ftp = calculate_ftp(specific_player)
    gp = len(specific_player)
    name = player
    all_stats[player] = Player(name, ppg, apg, trb, spg, bpg, fgapg, fgp, ftapg, ftp, gp, specific_player)
    
# Sanity Check
print('Stephen Curry 2017-18 Season Statistics:')
print("Points per Game:", "%.1f" % all_stats['Stephen Curry'].ppg)
print("Assists per Game:", "%.1f" % all_stats['Stephen Curry'].apg)
print("Rebounds per Game:", "%.1f" % all_stats['Stephen Curry'].rpg)

Stephen Curry 2017-18 Season Statistics:
Points per Game: 26.4
Assists per Game: 6.1
Rebounds per Game: 5.1


![stephen_curry](images/curry_stats.png)

In [250]:
# Team Representation
grp.head(PREVIEW_LEN)

gmDate gmTime  seasTyp teamAbbr  teamConf  teamDiv  \
gmDate   teamAbbr                                                         
20171017 BOS       20171017   0800        1      BOS         0        0   
         CLE       20171017   0800        1      CLE         0        2   
         GS        20171017   1030        1       GS         1        5   
         HOU       20171017   1030        1      HOU         1        4   
20171018 ATL       20171018   0830        1      ATL         0        1   
         BKN       20171018   0700        1      BKN         0        0   
         BOS       20171018   0730        1      BOS         0        0   
         CHA       20171018   0700        1      CHA         0        1   
         DAL       20171018   0830        1      DAL         1        4   
         DEN       20171018   0900        1      DEN         1        3   

                   teamLoc  teamRslt  teamMin  teamDayOff  teamPTS  teamAST  \
gmDate   teamAbbr                                                             
20171017 BOS             0         0      241           0       99       24   
         CLE             1         1      240           0      102       19   
         GS              1         0      241           0      121       34   
         HOU             0         1      239           0      122       28   
20171018 ATL             0         1      241           0      117       20   
         BKN             0         0      241           0      131       22   
         BOS             1         0      240           0      100       23   
         CHA             0         0      238           0       90       16   
         DAL             1         0      240           0      111       27   
         DEN             0         0      242           0       96       21   

                   teamTO  teamSTL  teamBLK  teamPF  teamFGA  teamFGM  \
gmDate   teamAbbr                                                       
20171017 BOS           12       11        4      24       88       36   
         CLE           17        3        4      25       83       38   
         GS            17        5        9      25       80       43   
         HOU           13        9        5      16       97       47   
20171018 ATL           13       11        6      18       94       48   
         BKN           20        7        2      25       94       45   
         BOS           15       12        2      27       91       39   
         CHA           17        4        3      15       73       29   
         DAL           16        5        4      19       86       38   
         DEN           22        4        3      18       77       36   

                   teamFG%  team2PA  team2PM  team2P%  team3PA  team3PM  \
gmDate   teamAbbr                                                         
20171017 BOS        0.4091       56       28   0.5000       32        8   
         CLE        0.4578       61       33   0.5410       22        5   
         GS         0.5375       50       27   0.5400       30       16   
         HOU        0.4845       56       32   0.5714       41       15   
20171018 ATL        0.5106       76       39   0.5132       18        9   
         BKN        0.4787       64       33   0.5156       30       12   
         BOS        0.4286       63       28   0.4444       28       11   
         CHA        0.3973       43       20   0.4651       30        9   
         DAL        0.4419       41       21   0.5122       45       17   
         DEN        0.4675       50       23   0.4600       27       13   

                   team3P%  teamFTA  teamFTM  teamFT%  teamORB  teamDRB  \
gmDate   teamAbbr                                                         
20171017 BOS        0.2500       25       19   0.7600        9       37   
         CLE        0.2273       25       21   0.8400        9       41   
         GS         0.5333       21       19   0.9048        6       35   
         HOU      

In [ ]:
# Model Params / Features Definition

In [ ]:
# Model Training

In [ ]:
# Scenarios